In [1]:
import cv2
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import os
import FILE_PATHS
from model import UNET
import masks_prediction

# Model load

In [2]:
# Hyperparameters
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 64
NUM_EPOCHS = 20
NUM_WORKERS = 4
IMAGE_HEIGHT = 112
IMAGE_WIDTH = 112
PIN_MEMORY = True
LOAD_MODEL = False

model_path = "../EchoNet-Dynamic/models/mask_model_old.pt"

In [3]:
MODEL = torch.load(model_path)
MODEL.eval()
print()

# Video preparation

- Get paths
- Extract frames and predict per frame
- Overlay predicted mask to frame
- Generate videos with overlayed frames

In [4]:
videos_path = f"{FILE_PATHS.NEW_DATA}/videos"
videos_filesnames = os.listdir(videos_path)

### Single file for visualization

(Excludes video generation)

In [5]:
sample_vid_path = "../EchoNet-Dynamic/videos/0X1A0A263B22CCD966.avi"

frames_list, predictions_list = masks_prediction.read_video_and_predict(
    sample_vid_path, model=MODEL
)

In [6]:
frame_list = []
for frame, mask in zip(frames_list, predictions_list):
    frame = masks_prediction.preprocess_raw_frame(frame)
    result_image = masks_prediction.overlay_mask_on_frame(frame, mask)
    frame_list.append(result_image)

# Full video generation

In [7]:
results_path = f"{FILE_PATHS.PREDICTIONS}/masks"
if not os.path.exists(results_path):
    os.makedirs(results_path)

predicted_videos = set(os.listdir(results_path))
video_extensions = set([".mp4", ".avi"])

In [8]:
for video_name in videos_filesnames:
    _, extension = os.path.splitext(video_name)
    if extension not in video_extensions:
        continue

    video_path = videos_path + "/" + video_name
    frames_list, predictions_list = masks_prediction.read_video_and_predict(
        video_path, model=MODEL
    )

    overlayed_frames = []
    for frame, mask in zip(frames_list, predictions_list):
        frame = masks_prediction.preprocess_raw_frame(frame)
        result_image = masks_prediction.overlay_mask_on_frame(frame, mask)
        overlayed_frames.append(result_image)

    masks_prediction.generate_video_from_list_of_frames(
        overlayed_frames, f"../EchoNet-Dynamic/results/masks/{video_name}", 20.0
    )

# Epochs evolution

In [44]:
epochs_path = f"{FILE_PATHS.PREDICTIONS}/masks/test_epochs"
if not os.path.exists(epochs_path):
    os.makedirs(epochs_path)

epochs_file_content = set(os.listdir(epochs_path))
epochs_frame_list = []
for i, filename in enumerate(epochs_file_content):
    _, extension = os.path.splitext(filename)
    if extension in video_extensions:
        continue

    epoch_frame_raw = cv2.imread(f"{epochs_path}/{filename}")
    font = cv2.FONT_HERSHEY_DUPLEX
    font_scale = 2
    font_color = (150, 255, 150)
    font_thickness = 4
    text_pos = (10, 50)
    epoch_frame = epoch_frame_raw.copy()
    epoch_text = f"Epoch {i+1}"
    cv2.putText(
        epoch_frame, epoch_text, text_pos, font, font_scale, font_color, font_thickness
    )
    epochs_frame_list.append(epoch_frame)
    # repeat first and last frame to make it longer
    if i == 0 or i == len(epochs_file_content) - 1:
        for i in range(10):
            epochs_frame_list.append(epoch_frame)

In [45]:
result_video_path = f"{epochs_path}/epochs_evolution.avi"
masks_prediction.generate_video_from_list_of_frames(
    epochs_frame_list, result_video_path, 10.0
)